# Navigation

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the first project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893).

## 1. Start the Environment

We begin by importing some necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [ ]:
# Only necessary when executing this notebook on the Udacity cloud
#!pip -q install ../python

In [ ]:
from unityagents import UnityEnvironment
import numpy as np
import os
import time

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Banana.app"`
- **Windows** (x86): `"path/to/Banana_Windows_x86/Banana.exe"`
- **Windows** (x86_64): `"path/to/Banana_Windows_x86_64/Banana.exe"`
- **Linux** (x86): `"path/to/Banana_Linux/Banana.x86"`
- **Linux** (x86_64): `"path/to/Banana_Linux/Banana.x86_64"`
- **Linux** (x86, headless): `"path/to/Banana_Linux_NoVis/Banana.x86"`
- **Linux** (x86_64, headless): `"path/to/Banana_Linux_NoVis/Banana.x86_64"`

For instance, if you are using a Mac, then you downloaded `Banana.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Banana.app")
```

In [ ]:
# Standalone machine
env = UnityEnvironment( os.path.join( os.environ['HOME'], 'Python/rl/udadrl/data/Banana_Linux/Banana.x86_64' ) ) # Visible (Best 4 testing)
#env = UnityEnvironment( os.path.join( os.environ['HOME'], 'Python/rl/udadrl/data/Banana_Linux_NoVis/Banana.x86_64' ) ) # Invisible (Best 4 training)

# Udacity cloud
#env = UnityEnvironment( file_name="/data/Banana_Linux_NoVis/Banana.x86_64" )

Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [ ]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

## 2. Examine the State and Action Spaces

The simulation contains a single agent that navigates a large environment.  At each time step, it has four actions at its disposal:
- `0` - walk forward 
- `1` - walk backward
- `2` - turn left
- `3` - turn right

The state space has `37` dimensions and contains the agent's velocity, along with ray-based perception of objects around agent's forward direction.  A reward of `+1` is provided for collecting a yellow banana, and a reward of `-1` is provided for collecting a blue banana. 

Run the code cell below to print some information about the environment.

In [ ]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents in the environment
print('Number of agents:', len(env_info.agents))

# number of actions
action_size = brain.vector_action_space_size
print('Number of actions:', action_size)

# examine the state space 
state = env_info.vector_observations[0]
print('States look like:', state)
state_size = len(state)
print('States have length:', state_size)

## 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action (uniformly) at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, as part of the project, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

In [ ]:
env_info = env.reset(train_mode=False)[brain_name] # reset the environment
state = env_info.vector_observations[0]            # get the current state
score = 0                                          # initialize the score
while True:
    action = np.random.randint(action_size)        # select an action
    env_info = env.step(action)[brain_name]        # send the action to the environment
    next_state = env_info.vector_observations[0]   # get the next state
    reward = env_info.rewards[0]                   # get the reward
    done = env_info.local_done[0]                  # see if episode has finished
    score += reward                                # update the score
    state = next_state                             # roll over the state to next time step
    if done:                                       # exit loop if episode finished
        break
    
print("Score: {}".format(score))

When finished, you can close the environment.

In [ ]:
env.close()

## 4. It's My Turn! - Train the agent!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [ ]:
from dql_agent import Agent
from math import tanh

#agent = Agent( state_size, action_size, seed = int( time.time() ) )
agent = Agent(state_size, action_size) #Create an agent with the default seed

# Helper functions
def get_time_string():
    return time.strftime('%Y%m%d%H%M%S')

def get_epsilon_i(num_episodes, epsilon_min = 0.01, evaluate = True): # evaluate is True by default coz tanh doesn't work well with Sarsa
        """ Getting a nice numpy array with epsilon values for every episode according to
        epsilon_i = epsilon_min+(1.0-epsilon_min)*(1-tanh(10*(i/num_episodes))) """
        if evaluate:
            epsilon = [1.0 / (i+1) for i in range(num_episodes+10)]
        else:
            epsilon = [epsilon_min+(1.0-epsilon_min)*(1-tanh(10*(i/num_episodes))) for i in range(num_episodes+10)]
        return epsilon
    

In [ ]:
scores = []  # List with all scores per episode
scores_100_mean = 0 # Mean score over the last 100 episodes

NUM_EPISODES = 2000
CRIT_SOLVED = 13 #How many Bananas must be collected to succeed?

epsilon = get_epsilon_i(NUM_EPISODES, evaluate = False)

for epc in range(NUM_EPISODES):

    env_info = env.reset(train_mode=True)[brain_name]  # reset the environment
    state = env_info.vector_observations[0]            # get the current state
    score = 0     # initialize the score
    

    start_time = time.time()

    while True:
        action = agent.act(state, epsilon[epc])        # select an action
        env_info = env.step(action)[brain_name]        # send the action to the environment
        next_state = env_info.vector_observations[0]   # get the next state
        reward = env_info.rewards[0]                   # get the reward
        done = env_info.local_done[0]                  # see if episode has finished
        agent.step(state, action, reward, next_state, done)
        score += reward                                # update the score
        state = next_state                             # roll over the state to next time step
        if done:                                       # exit loop if episode finished
            break

    scores.append(score)
    if len(scores) >= 100:
        scores_100_mean = np.mean( scores[-100:] )
        
    
    stop_time = time.time()

    print( '\rEpoch:{:>5}; Score: {:>5}; Epsilon: {:2.4f}; Mean (100): {:+3.2f}'.format( epc+1, score, epsilon[epc], scores_100_mean ), \
          end = ' ')
    if ( epc+1 ) % 100 == 0:       
        print('*')
        
    if (scores_100_mean > CRIT_SOLVED):
        print('#')
        print( 'Udacity criterion reached (Mean of recent 100 runs > 13), enviroment is considered as solved!')
        #print( 'My criterion reached (Mean of recent 100 runs > '+ str(CRIT_SOLVED) +'), enviroment is considered as solved!')
        break


In [ ]:
import matplotlib.pyplot as plt

plt.plot(scores)
plt.ylabel('Training scores')
plt.show()

In [ ]:
# Thx2: https://pytorch.org/tutorials/beginner/saving_loading_models.html

import torch

timestamp = get_time_string()
torch.save(agent.qnetwork_local.state_dict(), './output/' + timestamp + '_qnetwork_local_statedict_slvd_13.pth')
print('Timestamp is: ' + timestamp)

In [ ]:
env.close()

## 5. Watch the trained agent in action!

In [ ]:
from dql_agent import Agent

#agent = Agent(state_size, action_size, seed = int( time.time() ) )
agent = Agent(state_size, action_size) #Create an agent with the default seed

In [ ]:
import torch

FILENAME = '20200707113405_qnetwork_local_statedict_slvd_15.pth'

# FA The network from my GPU training run did not load
# Thx 2: https://stackoverflow.com/a/55759312

if torch.cuda.is_available():
    map_location=lambda storage, loc: storage.cuda()
    print('CUDA')
else:
    map_location='cpu'
    print('CPU')
    

# load the weights from file
agent.qnetwork_local.load_state_dict(torch.load( os.path.join('./output/', FILENAME), map_location=map_location))
agent.qnetwork_local.eval() #Set Network in evaluation mode - important!




In [ ]:
score_lst = []

for epc in range(3):

    env_info = env.reset(train_mode=False)[brain_name] # Reset & train_mode => False -> fast & True -> slow
    state = env_info.vector_observations[0]            # get the current state
    score = 0     # initialize the score

    start_time = time.time()

    while True:
        action = agent.act( state, eps = 0.01 )        # select an action & keep a bit randomness (eps)
        env_info = env.step(action)[brain_name]        # send the action to the environment
        next_state = env_info.vector_observations[0]   # get the next state
        reward = env_info.rewards[0]                   # get the reward
        done = env_info.local_done[0]                  # see if episode has finished
        score += reward                                # update the score
        state = next_state                             # roll over the state to next time step
        if done:                                       # exit loop if episode finished
            break

    stop_time = time.time()
    score_lst.append(score)
    print( '\rEpoch:{:>5}; Score: {:>5}; Execution time: {:.4f}'.format( epc+1, score, stop_time - start_time ), end = ' ' )
    
    if (epc+1) % 100 == 0:
        print('; {:2.2f} (mea); {:2.2f} (std)'.format(np.mean( score_lst[-100:] ), np.std( score_lst[-100:] )))

print( 'Achieved mean and standard deviation over {} test runs: {:2.2f} (mea) {:2.2f} (std) '\
      .format( ( epc+1 ), np.mean(score_lst), np.std(score_lst) ) )


In [ ]:
env.close()